In [50]:
import random
import numpy as np
from datetime import datetime, timedelta
import pandas as pd

Generate the Fake Ticket Dataset

In [51]:
#ensure that every time I run the script, it is the same random dataset

random.seed(42)
np.random.seed(42)

In [52]:
#Make the ticket generator

def generate_fake_tickets(n=500):
    #random lookup tables
    categories = ['Hardware', 'Software', 'Network', 'Access Request']
    priorities = ['Low','Medium','High','Critical']
    statuses = ['Open', 'In Progress', 'Resolved', 'Closed']
    sla_hours = {'Low': 72, 'Medium': 48, 'High': 24, 'Critical': 8} 
    '''how long it should take for ticket'''

    base_date = datetime(2024,1,1) #earliest creation date (year,month,day)
    data = []

    #Per ticket loop
    for i in range(n): # from n=500
        #Ticket creation and resolution
        created = base_date + timedelta(days=random.randint(0, 180), hours=random.randint(0, 23)) #random.randint(first day, last day) with 0 being january 1 2024
        priority = random.choice(priorities) #pick random priority
        sla = timedelta(hours=sla_hours[priority]) #pick sla based on random priority
        resolution_time = timedelta(hours=random.randint(1, sla_hours[priority] + 20)) #how long it actually took, which is why the +20 hrs
        resolved = created + resolution_time #actual date it was resolved
        now = datetime(2024, 5, 15)  # pretend today is May 15, 2024
        status = 'Resolved' if resolved < now else 'Open' #if the ticket is resolved mark resolved if not then mark open based on 'now' date
        sla_met = resolution_time <= sla #mark true or false if resolved in time or not

        #Items in all the columns, store ticket date
        data.append({
            'ticket_id': f'TKT-{1000 + i}', #the id of the ticket starting at 1000 increasing by 1
            'category': random.choice(categories), #pick from random catagories variable
            'priority': priority, 
            'created_at': created,
            'resolved_at': resolved if status == 'Resolved' else None, #if status is "Resolved" then add time it is resolved
            'status': status,
            'resolution_hours': resolution_time.total_seconds() / 3600, #get number of hours as a float
            'sla_hours': sla.total_seconds() / 3600, #get number of hours as a float
            'sla_met': sla_met
        })

    return pd.DataFrame(data) #put in dataframe, or I won't get anything lol

df = generate_fake_tickets() #not needed for code to work, just to work more easily with the rest so I do not have to rewrite that vairable name
df.to_csv("fake_tickets.csv", index=False)
print("✅ fake_tickets.csv created")

✅ fake_tickets.csv created


KPI Analysis & Reporting

In [53]:
#open file

df = pd.read_csv('fake_tickets.csv')
df

,ticket_id,category,priority,created_at,resolved_at,status,resolution_hours,sla_hours,sla_met
0,TKT-1000,Software,Low,2024-06-12 03:00:00,NaN,Open,36.0,72.0,True
1,TKT-1001,Hardware,Low,2024-02-27 04:00:00,2024-03-01 19:00:00,Resolved,87.0,72.0,False
2,TKT-1002,Hardware,Low,2024-05-31 13:00:00,NaN,Open,4.0,72.0,True
3,TKT-1003,Software,Low,2024-02-25 07:00:00,2024-02-28 07:00:00,Resolved,72.0,72.0,True
4,TKT-1004,Access Request,Critical,2024-06-15 22:00:00,NaN,Open,8.0,8.0,True
...,...,...,...,...,...,...,...,...,...
495,TKT-1495,Software,Low,2024-03-26 09:00:00,2024-03-27 13:00:00,Resolved,28.0,72.0,True
496,TKT-1496,Software,Critical,2024-01-31 23:00:00,2024-02-01 07:00:00,Resolved,8.0,8.0,True
497,TKT-1497,Access Request,High,2024-04-11 07:00:00,2024-04-12 02:00:00,Resolved,19.0,24.0,True
498,TKT-1498,Network,High,2024-04-29 17:00:00,2024-04-30 13:00:00,Resolved,20.0,24.0,True


In [54]:
#SLA Compliance Rate, what % resolved tickets meet SLA?

sla_compliance = df[df['status'] == 'Resolved']['sla_met'].mean()
print(f"SLA Compliance Rate: {sla_compliance:.2%}")

SLA Compliance Rate: 60.83%


In [55]:
#Average Resolution Time, how long did resolved tickets take on average?

avg_resolution = df[df['status'] == 'Resolved']['resolution_hours'].mean()
print(f'Average Resolution Time: {avg_resolution:.2f}')

Average Resolution Time: 29.42


In [56]:
#Ticket Volume by Category or Priority, how many tickets fall into each catagory?

category_count = df['category'].value_counts()
priority_count = df['priority'].value_counts()

print(f'Tickets by Category:',category_count)
print(f'\nTickets by Priority:',priority_count)

Tickets by Category: category
Access Request    135
Software          132
Network           130
Hardware          103
Name: count, dtype: int64

Tickets by Priority: priority
Low         131
High        128
Medium      123
Critical    118
Name: count, dtype: int64


In [57]:
#Summary

summary = {
    'SLA Compliance Rate': [sla_compliance],
    'Average Resolution Hours': [avg_resolution]
}
pd.DataFrame(summary).to_csv("summary_stats.csv", index=False)